In [1]:
## This notebook is to test our saved model pipeline on live data and then save the results

In [93]:
from sklearn.externals import joblib

dtc_file = 'dtc_pipeline_baseline_full_v2.pkl'
dtc = joblib.load(dtc_file)
type(dtc)

sklearn.pipeline.Pipeline

In [109]:
import pandas as pd
# test the pipeline on live data
modis_url = 'https://firms.modaps.eosdis.nasa.gov/data/active_fire/c6/csv/MODIS_C6_USA_contiguous_and_Hawaii_24h.csv'
live_modis = pd.read_csv(modis_url)

In [110]:
live_modis.satellite.value_counts()

T    254
A    168
Name: satellite, dtype: int64

In [111]:
live_modis.acq_date.value_counts()

2019-09-17    394
2019-09-18     28
Name: acq_date, dtype: int64

In [112]:
def process_live_data(original_df):
    """
    Pre processes live data to match pipeline expectations.
    """
    df = original_df.copy()
    #process satellite labels
    df['satellite'] = df['satellite'].replace({'T':'Terra', 'A': 'Aqua'})
    
    #process time features
    df['acq_time'] = (df['acq_time']//100)*60 +(df['acq_time']%100)
    df['timestamp'] = df.apply(lambda x: datetime.datetime.strptime(x['acq_date'],
                        '%Y-%m-%d') + datetime.timedelta(minutes=x['acq_time']), axis=1)
    df['month'] = df['timestamp'].dt.month
    df['week'] = df['timestamp'].dt.weekofyear
    df.drop(columns=['acq_date', 'acq_time'], inplace=True)
    return df

In [113]:
df = process_live_data(live_modis)

In [114]:
df.head()

,latitude,longitude,brightness,scan,track,satellite,confidence,version,bright_t31,frp,daynight,timestamp,month,week
0,18.930,-70.351,313.1,1.4,1.2,Terra,86,6.0NRT,283.9,20.1,N,2019-09-17 02:40:00,9,38
1,18.927,-70.365,302.0,1.4,1.2,Terra,46,6.0NRT,283.9,7.5,N,2019-09-17 02:40:00,9,38
2,19.754,-70.624,304.7,1.4,1.2,Terra,61,6.0NRT,292.0,7.8,N,2019-09-17 02:40:00,9,38
3,27.452,-98.116,311.2,1.6,1.2,Terra,81,6.0NRT,293.1,17.1,N,2019-09-17 04:25:00,9,38
4,27.453,-98.121,307.1,1.6,1.2,Terra,68,6.0NRT,292.9,12.4,N,2019-09-17 04:25:00,9,38


In [106]:
df.dtypes

latitude               float64
longitude              float64
brightness             float64
scan                   float64
track                  float64
satellite               object
confidence               int64
version                 object
bright_t31             float64
frp                    float64
daynight                object
acq_time_min             int64
timestamp       datetime64[ns]
month                    int64
week                     int64
dtype: object

In [11]:
dtc.predict(df)

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,

In [12]:
# success!
live_modis['fire'] = dtc.predict(df)

In [13]:
# this is what we will save into our database
results = live_modis[['latitude', 'longitude', 'fire']]

In [12]:
# now we will try to save the results to a db
import sqlite3

sl_conn = sqlite3.connect('fire_predictions_db.sqlite3')
sl_curs = sl_conn.cursor()

In [13]:
create_modus_table = """
CREATE TABLE modus(
    id INTEGER PRIMARY KEY,
    latitude NUMERIC(3,1),
    longitude NUMERIC(3,1),
    fire SMALLINT
)
"""


In [14]:
# try a table representation
# next create table in postgresql
# make a schema

create_modus_table = """
CREATE TABLE modus(
    id INTEGER PRIMARY KEY,
    latitude NUMERIC(6,2),
    longitude NUMERIC(6,2),
    fire SMALLINT
)
"""

drop_table = """
DROP TABLE modus
"""

In [16]:
# sl_curs.execute(drop_table)
sl_curs.execute(create_modus_table)

In [17]:
results.to_sql('modus', con=sl_conn, if_exists='replace')

In [18]:
# for row in results.values:
    attempted_insert = """
    INSERT INTO modus (latitude, longitude, fire)
    VALUES """+ str(tuple(row))
    sl_curs.execute(attempted_insert)

In [19]:
# check results
sl_curs.execute("SELECT * FROM modus LIMIT 10").fetchall()
# rows = sl_curs.fetchall()
 
# for row in rows:
#     print(row)

[(0, 31.148000000000003, -87.993, 0),
 (1, 37.236999999999995, -82.042, 0),
 (2, 39.483000000000004, -84.385, 0),
 (3, 38.69, -90.135, 0),
 (4, 41.183, -87.86200000000001, 0),
 (5, 41.628, -87.147, 0),
 (6, 42.68600000000001, -111.587, 0),
 (7, 47.92100000000001, -115.726, 1),
 (8, 26.889, -101.454, 0),
 (9, 30.5, -109.63799999999999, 0)]

In [20]:
results.head(10)

,latitude,longitude,fire
0,31.148,-87.993,0
1,37.237,-82.042,0
2,39.483,-84.385,0
3,38.690,-90.135,0
4,41.183,-87.862,0
5,41.628,-87.147,0
6,42.686,-111.587,0
7,47.921,-115.726,1
8,26.889,-101.454,0
9,30.500,-109.638,0


## Live flask testing code

In [119]:
ts = datetime.datetime.now()

print('timestamp: ', ts)
print('datatype: ', type(ts))

timestamp:  2019-09-18 09:59:47.124957
datatype:  <class 'datetime.datetime'>


In [120]:
len(str(-3555917140992459748))

20